In [1]:
# minN: cantidad de observaciones minima que debe tener una gvkey_hscode6 para entrar en el panel
minN = 30

In [2]:
measure = 'volumeteu'
period = 'Q'
source = 'NO'
hscode6 = True

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import statsmodels.api as sm
import json
import os
from get_plot_dict import get_plot_dict
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from datequarter import DateQuarter
import datetime
import math
import numpy as np
from sklearn.linear_model import LinearRegression

In [4]:
def get_sector_list(df, code2):
    """ de df, gets all the hscode6 que empiezan con code2 """
    hscode6_list = [x for x in map(str, df['hscode6'].unique()) if x.startswith(code2)]
    hscode6_array = [int(i) for i in hscode6_list]
    return hscode6_array


def get_panjiva():
    """ Data PANJIVA """
    df = pd.read_csv('portof(un)lading/panjivausimport_imputevalue_shippingtime_yq.csv')
    df = df[['gvkey', 'year', 'quarter', 'hscode6', 'shpcountry', 'valueofgoodsusd_new', 'volumeteu', 'weightkg']]
    df.drop(columns=['weightkg', 'valueofgoodsusd_new'], inplace=True)
    return df


def get_data_continent(df):
    df.loc[list(map(lambda country: country.startswith('Congo'), df['shpcountry'])), 'shpcountry'] = 'Congo'
    df.loc[list(map(lambda country: country.startswith('Micronesia'), df['shpcountry'])), 'shpcountry'] = 'Micronesia'
    df.loc[list(map(lambda country: country.startswith('Moldova'), df['shpcountry'])), 'shpcountry'] = 'Moldova'
    df.loc[list(map(lambda country: country.startswith('Micronesia'), df['shpcountry'])), 'shpcountry'] = 'Micronesia'
    df.loc[list(map(lambda country: country.startswith('Virgin Islands (U.S.)'), df['shpcountry'])), 'shpcountry'] = 'United States Virgin Islands'
    df.loc[list(map(lambda country: country.startswith('Virgin Islands (British)'), df['shpcountry'])), 'shpcountry'] = 'British Virgin Islands'
    df.loc[list(map(lambda country: country.startswith('Samoa'), df['shpcountry'])), 'shpcountry'] = 'Samoa'

    country_continent = pd.read_csv('country_continent.csv')
    df = df.merge(country_continent, left_on='shpcountry', right_on='COUNTRY', how='left', suffixes=None)  
    df['source'] = 'Rest of the World'
    df.loc[(df['CONTINENT'] == 'Asia', 'source')] = 'Rest of Asia'
    df.loc[(df['shpcountry'] == 'China', 'source')] = 'China'
    df.loc[(df['CONTINENT'] == 'Europe', 'source')] = 'Europe'
    return df


def get_data_sector(df, code6):
   
    df['hscode6_str'] = df['hscode6'].astype(str)
    df['hscode4_str'] = df['hscode6_str'].str.slice(stop=3)
    df['hscode4'] = df['hscode4_str'].astype(int)
    
    df = df[df['hscode6'].isin(code6)]
    
    df["qs_"] = list(map(DateQuarter,df["year"],df["quarter"]))
    df['qs'] = df['year'].astype(str) +'-Q'+ df['quarter'].astype(str) 
    df['date'] = pd.PeriodIndex(df['qs'], freq='Q').to_timestamp()
    return df

In [5]:
def get_data(df_panjiva, code2, measure='volumeteu', hscode6=True, period='Q', source='country', years=[2009,2019]):
    """ 
    measure = 'volumeteu', 'weightkg'
    hscode6 = True, False
    period = 'Y', 'Q'
    source = 'continent', 'country', 'NO'
    """
    
    code2_name = hscode2[code2]
    code6 = get_sector_list(df_panjiva, code2)
    
    df = get_data_sector(df_panjiva, code6)
    df = get_data_continent(df)    
    df.rename(columns={"COUNTRY": "country", "CONTINENT": "continent"}, inplace=True)
    
    df = df[df.year>=years[0]]
    df = df[df.year<=years[1]]
    
    period_name = lambda period: 'date' if period=='Q' else 'year'
    
    cols = ['gvkey', period_name(period), measure, "country"]
    
    if hscode6: cols = cols + ['hscode6']
    if source !='NO': cols = cols + [source]       
    
    df = df[cols]
    
    cols.remove('volumeteu')
    
    df = df.groupby(cols).sum() 
    df.reset_index(inplace=True)

    df = df[df.volumeteu!=0]
    df['code2'] = code2
    df.rename(columns={"volumeteu": "vol"}, inplace=True)
    
    return df


def get_filename(code2, measure, hscode6, period, source):
    """ Genera el nombre del archivo para guardar"""
    bol_hscode6 = lambda bol: '_code6' if int(hscode6) else ''
    bol_source = lambda source: '_' + source if source != 'NO' else ''
    bol_measure = lambda source: '_' + 'vol' if measure == 'volumeteu' else ''
    filename = 'panel' +bol_source(source)+bol_hscode6(hscode6)+'_'+period.lower()+bol_measure(measure)+'_'+code2+'.csv'
    return filename

In [6]:
def gen_df_vol():
    for code2 in tqdm(['87','64','85','94','63','62','61','59','95']):
        df = get_data(df_panjiva, code2, measure=measure, hscode6=hscode6, period=period, source=source, years=[2009,2019])
        df = df.groupby(['gvkey', 'hscode6','date', 'code2']).sum()
        df.reset_index(inplace=True)
        filename = get_filename(code2, measure, hscode6, period, source=source)
        df.to_csv(os.path.join('DATA', filename))

In [7]:
def get_df_vol():
    df = pd.DataFrame()
    for code2 in ['87','64','85','94','63','62','61','59','95']:
        filename = get_filename(code2, measure, hscode6, period, source=source)
        df_ = pd.read_csv(os.path.join('DATA', filename))  
        df_.drop(columns=['Unnamed: 0'], inplace=True)
        df = df.append(df_)
    return df

In [8]:
def get_data_code6(df, minN=30):
    """ Data gvkey_hscode6 con >=T periodos observados."""

    df["gvkey_hscode6"] = df['gvkey'].astype(str) +"_"+ df["hscode6"].astype(str)
    
    
    def panel_N(df_gvkey_hscode6, minN=30):
        """ Cantidad de Obs. para cada gvkey_hscode6."""

        series = df_gvkey_hscode6['gvkey_hscode6'].value_counts()
        panel = pd.DataFrame(series)
        panel.rename(columns={'gvkey_hscode6': "N"}, inplace=True)
        panel['gvkey_hscode6'] = panel.index

        panel[['gvkey', 'hscode6']] = panel['gvkey_hscode6'].str.split("_", expand=True)
        panel = panel[panel['N']>=minN]

        df = df_gvkey_hscode6[df_gvkey_hscode6['gvkey_hscode6'].isin(panel['gvkey_hscode6'])]

        return df
    
    
    df = panel_N(df, minN)
    
    df['month'] = pd.DatetimeIndex(df['date']).month
    df['quarter']= df['month'].map({1:1, 4:2, 7:3, 10:4})
    
    dates = list(df['date'].unique())
    dates.sort()
    df_dates = pd.DataFrame(dates, columns=['fecha'])
    df_dates['t'] = df_dates.index
    fechas_t = {k:v for (k,v) in zip(df_dates['fecha'], df_dates['t'])}

    df['period']= df['date'].map(fechas_t)
    
    df[['1', '2', '3', '4']] = pd.get_dummies(df['quarter'])
    df['vol_log'] = np.log2(df['vol']+1)
    df['period_log'] = np.log2(df['period']+1)
    
    return df

In [9]:
def get_sigmaR(df):
    X = df[['period_log', '1', '2', '3', '4']]
    y = df['vol_log']
    reg = LinearRegression().fit(X, y)
    df['vol_log_hat'] = reg.predict(X)
    df['R'] = df['vol_log'] - df['vol_log_hat']
    
    R = np.array(df['R'])

    meanR = np.zeros_like(R)
    meanR[0] = R[0]
    meanR[1] = np.mean(R[0:2])
    meanR[2] = np.mean(R[0:3])
    meanR[3] = np.mean(R[0:4])
    for t in range(4, len(R)):
        meanR[t] = np.mean(R[t-3:t+1])    

    sumR = np.zeros_like(R)
    sumR[0] = R[0]
    sumR[1] = np.sum(R[0:2])
    sumR[2] = np.sum(R[0:3])
    sumR[3] = np.sum(R[0:4])
    for t in range(4, len(R)):
        sumR[t] = np.sum(R[t-3:t+1])  

    sigmaR = [math.sqrt(((s-m)**2)/4) for (s,m) in zip(sumR, meanR)]

    df['sigmaR'] = sigmaR
    return df

In [30]:
def gen_df_sigma():
    
    df = pd.DataFrame()
    for j in tqdm(list_code6):
        df_ = df_code6[df_code6['gvkey_hscode6']==j].sort_values(by=['period'])
        df_ = df_[3:]
        df = df.append(get_sigmaR(df_))

    df_sigma = df.copy()
    return df_sigma

In [11]:
def get_df_sigma():
    df_sigma = pd.read_csv(os.path.join('DATA', 'df_sigma.csv'))
    df_sigma.drop(columns=['Unnamed: 0'], inplace=True)
    print(df_sigma.shape)
    return df_sigma

In [12]:
def get_df_country_vol():
    df = pd.DataFrame()
    for code2 in tqdm(['87','64','85','94','63','62','61','59','95']): 
        df_ = get_data(df_panjiva, code2, measure=measure, hscode6=hscode6, period=period, source=source, years=[2009,2019])
        df = df.append(df_)
    return df

In [13]:
def get_list_code6():
    """ 
    La lista `gvkey_hscode6` tiene el volumeteu total para cada (gvkey_hscode6, date) 
    con mas (o igual) de 30 periodos de importacion (>=minN). 
    """
    df_sigma = pd.read_csv(os.path.join('DATA', 'df_sigma.csv'))
    list_code6 = list(df_sigma['gvkey_hscode6'].unique())
    return list_code6

In [28]:
def gen_df_countries():

    list_code6 = get_list_code6()
    df = get_df_country_vol()

    df["gvkey_hscode6"] = df['gvkey'].astype(str) +"_"+ df["hscode6"].astype(str)
    df = df[df['gvkey_hscode6'].isin(list_code6)]
    
    # ['Hong Kong', 'Taiwan'] == 'China'
    df.loc[df['country'] == "Taiwan", 'country'] = 'China'
    df.loc[df['country'] == "Hong Kong", 'country'] = 'China'
    
    # agrupa importaciones del mismo pais en um mismo quarter
    cols = list(df.columns) 
    cols.remove('vol')
    df = df.groupby(cols).sum() 
    df.reset_index(inplace=True)
    
    df['vol%'] = 100 * df['vol'] / df.groupby(['gvkey_hscode6', 'date'])['vol'].transform(sum)
    
    return df

In [29]:
def get_df_countries():
    df_countries = pd.read_csv(os.path.join('DATA', 'df_countries.csv'))
    df_countries.drop(columns=['Unnamed: 0'], inplace=True)
    return df_countries

In [16]:
def gen_df_source1():

    df = df_countries.copy()
    df.drop(columns=['gvkey', 'hscode6', 'code2'], inplace=True)
    df.rename(columns={'country': 'source1'}, inplace=True)
    df.reset_index(inplace=True, drop=True)

    df_source1 = df.loc[df.groupby(['gvkey_hscode6', 'date'])['vol'].idxmax()] #, 'vol%'
    return df_source1

In [17]:
def gen_df_panel():
    df_panel = pd.merge(df_sigma, df_source1, how="left", on=['date', 'gvkey_hscode6'])
    df_panel.drop(columns=['vol_y'], inplace=True)
    df_panel.rename(columns={"vol_x": "vol"}, inplace=True)
    return df_panel

In [18]:
f = open('hscode2.json')
hscode2 = json.load(f) 
f.close()

In [19]:
%%time
df_panjiva = get_panjiva()

Wall time: 7.46 s


In [20]:
%%time
df_vol = gen_df_vol()

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:53<00:00, 12.61s/it]

Wall time: 1min 53s


In [21]:
%%time
df_vol = get_df_vol()

Wall time: 594 ms


In [22]:
%%time
df_code6 = get_data_code6(df_vol, minN)

Wall time: 1.55 s


In [23]:
list_code6 = list(df_code6['gvkey_hscode6'].unique())

In [31]:
df_sigma = gen_df_sigma()

100%|██████████████████████████████████████████████████████████████████████████████| 5065/5065 [02:32<00:00, 33.30it/s]


In [32]:
df_sigma.to_csv(os.path.join('DATA', 'df_sigma.csv'))

In [33]:
df_sigma = get_df_sigma()

(173204, 18)


In [34]:
%%time
df_countries = gen_df_countries()

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:46<00:00, 11.86s/it]


Wall time: 1min 48s


In [35]:
df_countries.to_csv(os.path.join('DATA', 'df_countries.csv'))

In [36]:
df_countries = get_df_countries()

In [37]:
df_source1 = gen_df_source1()

In [38]:
df_panel = gen_df_panel()

In [39]:
df_panel.to_csv(os.path.join('DATA', 'panel.csv'))

In [ ]:
# df50 = df_countries[df_countries['vol%']==50].copy()
# df50

In [ ]:
# dictio = dict()
# indices = list(df50['gvkey_hscode6'].unique())
# for i in indices:
#     for date in df50[df50['gvkey_hscode6']==i]['date']:
#         df_ = df50[np.logical_and(df50['gvkey_hscode6']==i, df50['date']==date)]
#         countries = list(df_['country'])
#         dictio[(i, date)] = countries

In [ ]:
# df_source1 = df_countries.loc[df_countries.groupby(['gvkey', 'hscode6', 'date'])['vol'].idxmax()]
# df_source1.rename(columns={'country': 'source1'}, inplace=True)
# df_source1.drop(columns=['gvkey', 'hscode6', 'vol', 'code2', 'vol%'], inplace=True)

In [ ]:
# key1 = '1300_870323'
# key2 = '2009-04-01'
# for key1, key2 in tqdm(dictio):
#     lista = dictio[(key1, key2)]

#     if len(lista)==1: continue

#     df = df_source1[df_source1['gvkey_hscode6']==key1]
    
#     pos = list(df['date']).index(key2)
#     pos_cerca = pos-1
#     if pos==0: pos_cerca = 1
    
#     antiguo = list(df['source1'])[pos]
#     nuevo = list(df['source1'])[pos_cerca]

#     if nuevo in lista:
#         df_source1[df_source1['gvkey_hscode6']==key1].iloc[pos]['source1'] = nuevo
